<a href="https://colab.research.google.com/github/GordonHeg/Machine-Learning-Group-Assignment-Transfer-Learning-/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1


In [2]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1"

# List all files and directories in the dataset folder
for root, dirs, files in os.walk(dataset_path):
    print(f"Directory: {root}")
    for dir_name in dirs:
        print(f"  Subdirectory: {dir_name}")
    # for file_name in files:
    #     print(f"  File: {file_name}")


Directory: /root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1
  Subdirectory: meta
  Subdirectory: train
  Subdirectory: Test
  Subdirectory: Train
  Subdirectory: Meta
  Subdirectory: test
Directory: /root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1/meta
Directory: /root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1/train
  Subdirectory: 35
  Subdirectory: 13
  Subdirectory: 40
  Subdirectory: 7
  Subdirectory: 25
  Subdirectory: 1
  Subdirectory: 34
  Subdirectory: 16
  Subdirectory: 19
  Subdirectory: 37
  Subdirectory: 21
  Subdirectory: 18
  Subdirectory: 15
  Subdirectory: 17
  Subdirectory: 3
  Subdirectory: 10
  Subdirectory: 27
  Subdirectory: 22
  Subdirectory: 9
  Subdirectory: 8
  Subdirectory: 38
  Subdirectory: 28
  Subdirectory: 30
  Subdirectory: 33
  Subdirectory: 12
  Subdirectory: 24
  Subdirectory: 32
  Subdirectory: 36
  Subdirectory: 14
  Subdirecto

In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# Define the path to your training directory
train_dir = '/root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1/train'

# List all subdirectories (which correspond to different classes)
all_classes = os.listdir(train_dir)

# Sort the class names alphabetically
sorted_classes = sorted(all_classes, key=lambda x: int(x))

# Select the first 10 classes after sorting
selected_classes = sorted_classes[:10]

print(selected_classes)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [4]:
# Initialize lists to store images and labels
images = []
labels = []

# Loop through the selected classes to load images and their corresponding labels
for class_id, class_name in enumerate(selected_classes):
    class_dir = os.path.join(train_dir, class_name)
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        try:
            img = Image.open(img_path).resize((224, 224))  # Resize to (224, 224) for VGG16
            img = np.array(img)  # Convert image to numpy array

            if img.shape[2] == 4:  # Handle RGBA images if any
                img = img[:, :, :3]

            images.append(img)
            labels.append(class_id)  # Use class_id as label
        except Exception as e:
            continue

# Convert lists to numpy arrays for use in the model
X = np.array(images)
y = np.array(labels)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Optionally, normalize the images (scale pixel values to [0, 1] range)
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0

In [ ]:
# Assuming the test data is also in a directory with the same structure
test_dir = '/root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1/test'

test_images = []
test_labels = []

# Loop through the selected classes to load images and their corresponding labels
for class_id, class_name in enumerate(selected_classes):
    class_dir = os.path.join(test_dir, class_name)
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        try:
            img = Image.open(img_path).resize((224, 224))  # Resize to (224, 224) for VGG16
            img = np.array(img)  # Convert image to numpy array

            if img.shape[2] == 4:  # Handle RGBA images if any
                img = img[:, :, :3]

            test_images.append(img)
            test_labels.append(class_id)  # Use class_id as label
        except Exception as e:
            continue

# Convert to numpy arrays
X_test = np.array(test_images)
y_test = np.array(test_labels)

# Normalize test data
X_test = X_test.astype('float32') / 255.0

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # 10 classes for classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate on the test set
test_predictions = model.predict(X_test)
test_accuracy = accuracy_score(y_test, np.argmax(test_predictions, axis=1))

print(f"Test Accuracy: {test_accuracy:.4f}")
